In [ ]:
!pip install langchain langchain-community langchain-core pypdf faiss-cpu sentence-transformers langchain_google_genai langchain_openai langchain_chroma

# 모델 불러오기

In [ ]:
import os
from google.colab import userdata # 코랩 Secrets 기능 사용을 위한 임포트
from langchain_google_genai import ChatGoogleGenerativeAI
# 코랩 Secrets에서 GOOGLE_API_KEY를 가져오기
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'GOOGLE_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    exit() # API 키가 없으면 스크립트 종료
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
print(f"Selected LLM: {llm.__class__.__name__}")

GOOGLE_API_KEY가 환경 변수로 설정되었습니다.
Selected LLM: ChatGoogleGenerativeAI


# ConversationBufferMemory

*   메세지 저장, 추출 가능



In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory()
memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)


<ipython-input-2-52b3aa27d154>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [ ]:
# 메모리에 저장된 대화 내용 확인_'history' 키에 저장됨
memory.load_memory_variables({})

{'history': 'Human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?\nAI: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?'}

In [ ]:
# inputs: dictionary(key: "human" or "ai", value: 질문)
# outputs: dictionary(key: "ai" or "human", value: 답변)
memory.save_context(
    inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
    outputs={
        "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
    },
)

In [ ]:
# 2개의 대화 저장
memory.save_context(
    inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
    outputs={
        "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)
memory.save_context(
    inputs={"human": "인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?"},
    outputs={
        "ai": "본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다."
    },
)


In [ ]:
# history에 저장된 대화 기록 확인
print(memory.load_memory_variables({})["history"])

Human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?
AI: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?
Human: 네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?
AI: 감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다.
Human: 사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?
AI: 업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다.
Human: 인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?
AI: 본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다.


In [ ]:
# 추가로 2개의 대화 저장
memory.save_context(
    inputs={"human": "정보를 모두 입력했습니다. 다음 단계는 무엇인가요?"},
    outputs={
        "ai": "입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요."
    },
)
memory.save_context(
    inputs={"human": "모든 절차를 완료했습니다. 계좌가 개설된 건가요?"},
    outputs={
        "ai": "네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!"
    },
)


In [ ]:
# history에 저장된 대화 기록 확인
print(memory.load_memory_variables({})["history"])

Human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?
AI: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?
Human: 네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?
AI: 감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다.
Human: 사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?
AI: 업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다.
Human: 인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?
AI: 본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다.


In [ ]:
#return_messages=True로 설정

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

memory.save_context(
    inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
    outputs={
        "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
    },
)

memory.save_context(
    inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
    outputs={
        "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)


In [ ]:
# history에 저장된 대화 기록 확인_HumanMessage, AIMessage
memory.load_memory_variables({})["history"]


[HumanMessage(content='안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다.', additional_kwargs={}, response_metadata={})]

## 체인에 적용

In [ ]:
from langchain.chains import ConversationChain

# LLM 모델 생성
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
# ConversationChain 생성
conversation = ConversationChain(
    llm=llm,
    # ConversationBufferMemory 사용
    memory=ConversationBufferMemory(),
)

In [ ]:
# 대화 시작
response = conversation.predict(
    input="안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
)
print(response)

안녕하세요! 비대면 계좌 개설은 요즘 굉장히 편리해졌습니다.  하지만 어떤 은행을 이용하시고 싶으신지에 따라 방법이 조금씩 다를 수 있어요.  일반적으로 다음과 같은 단계를 거치게 됩니다.

* **은행 선택:**  먼저 어떤 은행에서 계좌를 개설할지 결정해야 합니다.  각 은행마다 비대면 계좌 개설 서비스의 편의성, 제공되는 서비스, 수수료 등이 다르기 때문에, 자신에게 맞는 은행을 선택하는 것이 중요합니다.  예를 들어, 특정 금융 상품에 관심이 있다면 그 상품을 제공하는 은행을 선택하는 것이 좋겠죠.  또는, 모바일 앱의 사용 편의성을 중요하게 생각하실 수도 있고요.

* **은행 앱 또는 웹사이트 접속:**  선택하신 은행의 모바일 앱 또는 웹사이트에 접속합니다.  대부분의 은행은 비대면 계좌 개설을 위한 전용 페이지를 제공합니다.

* **필요 정보 입력:**  개설하고자 하는 계좌 종류 (예: 입출금 계좌, 적금, 예금 등)를 선택하고,  본인 확인을 위한 정보 (주민등록번호, 휴대폰 번호, 이메일 주소 등)와 개인 정보 (주소, 직업 등)를 입력해야 합니다.  정확한 정보를 입력하는 것이 매우 중요합니다.

* **본인 인증:**  대부분의 은행은 본인 인증 절차를 거칩니다.  이는  휴대폰 인증,  아이핀(i-PIN) 인증, 또는 공인인증서를 이용한 인증 등 다양한 방법으로 진행됩니다.  어떤 방법을 사용해야 하는지는 선택하신 은행의 안내에 따라야 합니다.

* **계좌 개설 신청:**  모든 정보 입력과 본인 인증이 완료되면, 계좌 개설을 신청합니다.

* **필요 서류 제출 (경우에 따라):**  일부 은행에서는 추가적인 서류 제출을 요구할 수 있습니다.  예를 들어, 소득 증명 서류나 신분증 사본 등이 필요할 수 있습니다.  이 부분은 은행마다 다르므로, 해당 은행의 안내를 확인해야 합니다.

* **계좌 개설 완료:**  신청이 완료되면, 은행 측에서 계좌 개설 여부를 알려줍니다.  계좌가 개설되면 계좌번호와 비밀번호 등의 정보를 확인할 

In [ ]:
# 이전 대화내용을 불렛포인트로 정리해 달라는 요청을 보냅니다.
response = conversation.predict(
    input="이전 답변을 불렛포인트 형식으로 정리하여 알려주세요."
)
print(response)


네, 이전 답변을 다음과 같이 요약하여 불렛포인트 형식으로 정리해 드리겠습니다.

**비대면 계좌 개설 절차:**

* **은행 선택:**  본인에게 맞는 은행 (서비스 편의성, 제공 서비스, 수수료 등 고려)을 선택합니다.
* **은행 앱/웹사이트 접속:**  선택한 은행의 모바일 앱 또는 웹사이트에 접속합니다.
* **필요 정보 입력:** 계좌 종류, 개인 정보 (주민등록번호, 주소, 직업 등)를 정확하게 입력합니다.
* **본인 인증:** 휴대폰 인증, 아이핀(i-PIN) 인증, 공인인증서 등을 이용한 본인 인증을 진행합니다.
* **계좌 개설 신청:** 모든 정보 입력 및 본인 인증 후 계좌 개설을 신청합니다.
* **필요 서류 제출 (경우에 따라):**  소득 증명 서류, 신분증 사본 등 추가 서류 제출이 필요할 수 있습니다.
* **계좌 개설 완료:** 은행에서 계좌 개설 완료 여부를 알려줍니다.  계좌번호와 비밀번호 등을 확인합니다.


**주의사항:**

* 개인정보 보호에 유의하여 안전한 환경에서 진행해야 합니다.
* 각 은행별 절차가 다를 수 있으므로,  선택하신 은행의 안내를 반드시 확인해야 합니다.


이 요약이 도움이 되셨으면 좋겠습니다.  어떤 은행을 이용하실 계획인지 알려주시면 더욱 자세한 정보를 제공해 드릴 수 있습니다.


# ConversationBufferWindowMemory

*  최근 k개의 상호작용 저장



In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, return_messages=True)

memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)
memory.save_context(
    inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
    outputs={
        "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
    },
)
memory.save_context(
    inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
    outputs={
        "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)
memory.save_context(
    inputs={"human": "인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?"},
    outputs={
        "ai": "본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "정보를 모두 입력했습니다. 다음 단계는 무엇인가요?"},
    outputs={
        "ai": "입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요."
    },
)
memory.save_context(
    inputs={"human": "모든 절차를 완료했습니다. 계좌가 개설된 건가요?"},
    outputs={
        "ai": "네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!"
    },
)


<ipython-input-2-675159ca6b28>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, return_messages=True)


In [ ]:
# 대화 기록 확인
memory.load_memory_variables({})["history"]

[HumanMessage(content='정보를 모두 입력했습니다. 다음 단계는 무엇인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='모든 절차를 완료했습니다. 계좌가 개설된 건가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!', additional_kwargs={}, response_metadata={})]

## 체인에 적용

In [ ]:
from langchain.chains import ConversationChain
# ConversationChain 생성
conversation = ConversationChain(
    llm=llm,
    # ConversationBufferWindowMemory 사용
    memory=ConversationBufferWindowMemory(k=2),
)

In [ ]:
# 대화 시작
response = conversation.predict(
    input="침엽수와 활엽수의 차이에 대해 알려줘"
)
print(response)

네, 침엽수와 활엽수의 차이점에 대해 다시 한번 설명해 드리겠습니다.  이전에 설명드린 내용을 보다 간결하게 정리하고, 추가적인 정보도 함께 제공하겠습니다.

**핵심적인 차이점:**

* **잎의 모양:** 이것이 가장 큰 차이점입니다. 침엽수는 바늘 모양 또는 비늘 모양의 잎을 가지는 반면, 활엽수는 넓고 납작한 잎을 가집니다.  침엽수의 잎은 보통 상록성이지만, 낙엽침엽수도 존재합니다 (예: 메타세쿼이아). 활엽수는 낙엽성인 경우가 많지만, 상록활엽수도 있습니다 (예: 동백나무, 가시나무).

* **구과(솔방울):** 침엽수는 대부분 구과라는 목질의 열매를 맺어 씨앗을 보호합니다. 활엽수는 구과를 맺지 않고 다양한 형태의 열매를 맺습니다 (예: 도토리, 단풍나무의 날개 열매, 콩깍지 등).

* **생장 속도:** 일반적으로 침엽수는 활엽수보다 생장 속도가 느립니다.

* **목재:** 침엽수의 목재는 일반적으로 활엽수보다 가볍고 부드러우며, 활엽수의 목재는 더 단단하고 무거운 경향이 있습니다.  하지만 이는 절대적인 기준이 아니며, 종에 따라 다양한 특징을 보입니다.

* **수형(나무의 전체적인 모양):** 침엽수는 원뿔 모양의 수형을 갖는 경우가 많지만, 활엽수는 종에 따라 수형이 매우 다양합니다.


**세부적인 차이점 (이전 설명에서 추가 및 보완):**

* **수분 증발:** 침엽수의 바늘 모양 또는 비늘 모양 잎은 표면적이 작아 수분 증발을 최소화합니다. 반면 활엽수의 넓은 잎은 광합성에는 유리하지만 수분 증발이 많습니다. 이것이 낙엽수의 겨울철 낙엽 현상의 주요 원인 중 하나입니다.

* **광합성:** 활엽수의 넓은 잎은 침엽수보다 광합성 효율이 높을 수 있습니다. 하지만 빛의 양, 온도, 수분 등 환경 조건에 따라 달라질 수 있습니다.

* **서식지:** 침엽수는 추운 기후에 잘 적응했지만, 활엽수는 다양한 기후에서 자랄 수 있습니다.  하지만 이 또한 절대적인 것은 아닙니다. 예를 들어, 일부 침엽수는 열대 지방에서도 자랍니다.


In [ ]:
# 두번째 대화_첫번째 대화 내용과 관련된 질문
response = conversation.predict(
    input="잎의 모양에 대해 다시 한 번 알려줘"
)
print(response)

네, 잎의 모양에 대해 다시 설명해 드리겠습니다.  침엽수와 활엽수의 가장 큰 차이점 중 하나가 바로 잎의 모양입니다.

**침엽수:**  침엽수의 잎은 대부분 **바늘 모양** 또는 **비늘 모양**입니다.

* **바늘 모양:**  말 그대로 바늘처럼 가늘고 길며 끝이 뾰족한 모양입니다. 소나무, 전나무, 잣나무 등이 대표적인 예입니다.  이러한 바늘 모양의 잎은 표면적이 작아 수분 증발을 최소화하는 데 유리합니다.  추운 기후나 건조한 기후에 적응한 결과라고 볼 수 있습니다.

* **비늘 모양:**  바늘 모양보다는 훨씬 짧고 납작하며, 비늘처럼 겹쳐져 있습니다.  측백나무, 향나무 등이 이러한 잎 모양을 가지고 있습니다.  비늘 모양의 잎 역시 수분 손실을 줄이는 데 효과적입니다.

**활엽수:** 활엽수의 잎은 **넓고 납작한 모양**을 하고 있습니다.  이 때문에 침엽수에 비해 광합성을 위한 표면적이 훨씬 넓습니다.  하지만 그만큼 수분 증발이 많아지기 때문에, 낙엽수의 경우 겨울철에는 잎을 떨어뜨려 수분 손실을 막습니다.  활엽수의 잎 모양은 종류에 따라 매우 다양합니다.  예를 들어, 단풍나무는 손바닥 모양의 잎을 가지고 있고, 참나무는 둥글거나 타원형의 잎을 가지고 있으며, 은행나무는 부채꼴 모양의 잎을 가지고 있습니다.  잎 가장자리 또한 톱니 모양, 밋밋한 모양 등 다양합니다.


요약하자면, 침엽수는 바늘 모양이나 비늘 모양의 좁고 단단한 잎을 가지는 반면, 활엽수는 넓고 납작한 잎을 가지며 그 형태는 매우 다양합니다. 이러한 잎의 모양 차이는 각각의 나무가 살아가는 환경에 대한 적응의 결과입니다.  더 궁금한 점이 있으면 언제든지 질문해주세요.


In [ ]:
# 세번째 대화_첫번째 대화는 지워짐
response = conversation.predict(
    input="대한민국의 수도는?"
)
print(response)

대한민국의 수도는 서울입니다.


In [ ]:
# 네번째 대화_세번째 대화의 답변을 이용한 질문
response = conversation.predict(
    input="이전에 답변한 도시의 인구수는?"
)
print(response)

죄송합니다. 서울의 인구수에 대한 정보는 제가 접근할 수 있는 정보에 포함되어 있지 않습니다.  실시간 데이터를 요구하는 질문에는 답변드리기 어렵습니다.  인터넷 검색을 통해 서울의 인구수를 확인하시는 것이 가장 정확할 것입니다.


In [ ]:
# 다섯번째 대화_첫번째 대화 내용과 관련된 질문(첫번째 질문은 저장되어 있지 않아 이전과 다른 답변이 나옴)
response = conversation.predict(
    input="잎의 모양에 대해 다시 한 번 알려줘"
)
print(response)

죄송합니다.  "잎의 모양"에 대한 질문이 너무 포괄적입니다. 어떤 식물의 잎의 모양에 대해 질문하시는 건가요?  예를 들어, "단풍나무 잎의 모양은 어떻습니까?" 또는 "소나무 잎의 모양은 어떻습니까?" 와 같이 구체적인 식물의 이름을 말씀해주시면 그 잎의 모양에 대해 설명해 드릴 수 있습니다.  다양한 식물의 잎은 크기, 모양, 가장자리, 맥 등 매우 다양한 특징을 가지고 있기 때문에, 더 구체적인 정보가 필요합니다.


# ConversationTokenBufferMemory

*   토큰 길이를 사용하여 대화내용을 플러시(flush)할 시기 결정



In [ ]:
from langchain.memory import ConversationTokenBufferMemory
# 메모리 설정
memory = ConversationTokenBufferMemory(
    llm=llm, max_token_limit=150, return_messages=True  # 최대 토큰 길이를 150개로 제한
)

<ipython-input-8-648a433bc482>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


In [ ]:
memory.save_context(
    inputs={
        "human": "안녕하세요, 저는 최근에 여러분 회사의 공작 기계를 구매했습니다. 설치 방법을 알려주실 수 있나요?"
    },
    outputs={
        "ai": "안녕하세요! 구매해 주셔서 감사합니다. 해당 기계 모델 번호를 알려주시겠어요?"
    },
)
memory.save_context(
    inputs={"human": "네, 모델 번호는 XG-200입니다."},
    outputs={
        "ai": "감사합니다. XG-200 모델의 설치 안내를 도와드리겠습니다. 먼저, 설치할 장소의 전원 공급 상태를 확인해주세요. 기계는 220V 전원이 필요합니다."
    },
)
memory.save_context(
    inputs={"human": "전원은 확인했습니다. 다음 단계는 무엇인가요?"},
    outputs={
        "ai": "좋습니다. 다음으로, 기계를 평평하고 안정된 바닥에 배치해 주세요. 이후, 제공된 사용자 매뉴얼에 따라 케이블 연결을 진행해 주시기 바랍니다."
    },
)
memory.save_context(
    inputs={"human": "연결은 어떻게 하나요?"},
    outputs={
        "ai": "매뉴얼의 5페이지를 참조해 주세요. 케이블 연결에 관한 상세한 지침이 있습니다. 이 과정에서 어려움이 있으시면 추가적으로 도와드리겠습니다."
    },
)
memory.save_context(
    inputs={"human": "설치가 완료되면 어떻게 해야 하나요?"},
    outputs={
        "ai": "설치가 완료되면, 전원을 켜고 초기 구동 테스트를 진행해 주시기 바랍니다. 테스트 절차는 매뉴얼의 10페이지에 설명되어 있습니다. 만약 기계에 이상이 있거나 추가적인 지원이 필요하시면 언제든지 연락 주시기 바랍니다."
    },
)
memory.save_context(
    inputs={"human": "감사합니다, 도움이 많이 되었어요!"},
    outputs={
        "ai": "언제든지 도와드릴 준비가 되어 있습니다. 추가적인 질문이나 지원이 필요하시면 언제든지 문의해 주세요. 좋은 하루 되세요!"
    },
)


In [ ]:
# 대화내용 확인
memory.load_memory_variables({})["history"]

[AIMessage(content='설치가 완료되면, 전원을 켜고 초기 구동 테스트를 진행해 주시기 바랍니다. 테스트 절차는 매뉴얼의 10페이지에 설명되어 있습니다. 만약 기계에 이상이 있거나 추가적인 지원이 필요하시면 언제든지 연락 주시기 바랍니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='감사합니다, 도움이 많이 되었어요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='언제든지 도와드릴 준비가 되어 있습니다. 추가적인 질문이나 지원이 필요하시면 언제든지 문의해 주세요. 좋은 하루 되세요!', additional_kwargs={}, response_metadata={})]

## 체인에 적용

In [ ]:
from langchain.chains import ConversationChain
# ConversationChain 생성
conversation = ConversationChain(
    llm = llm,
    # ConversationTokenBufferMemory 사용
    memory=ConversationTokenBufferMemory(llm=llm, max_token_limit=500),
)

In [ ]:
# 대화 시작
conversation.predict(
    input="침엽수와 활엽수의 차이에 대해 알려줘"
)

'침엽수와 활엽수의 차이점은 여러 가지가 있지만, 가장 큰 차이점은 바로 잎의 모양과 구조입니다.  쉽게 말해, 침엽수는 바늘 모양 또는 비늘 모양의 잎을 가지고 있고, 활엽수는 넓고 평평한 잎을 가지고 있습니다.\n\n좀 더 자세히 설명해 드리자면:\n\n**침엽수 (Conifers):**\n\n* **잎의 모양:** 바늘 모양, 비늘 모양의 잎을 가지고 있습니다.  소나무의 잎처럼 길고 가늘거나, 편백나무처럼 작고 비늘 모양인 경우도 있습니다.  잎이 좁고 단단하며, 수분 증발을 최소화하기 위해 두꺼운 큐티클 층을 가지고 있습니다.\n* **잎의 수명:** 대부분 상록수이기 때문에 잎이 1년 내내 나무에 달려 있습니다.  하지만 일부 낙엽침엽수도 존재합니다. (예: 낙엽송)\n* **구과 (Cone):** 대부분 구과(솔방울)를 열매로 맺습니다.  구과 안에는 씨앗이 들어 있습니다.\n* **목재:** 일반적으로 단단하고 강하며, 건축재, 가구재 등으로 많이 사용됩니다.\n* **예시:** 소나무, 전나무, 잣나무, 삼나무, 편백나무, 낙엽송 등\n\n\n**활엽수 (Broadleaf trees):**\n\n* **잎의 모양:** 넓고 평평한 잎을 가지고 있습니다.  잎의 가장자리는 톱니 모양, 밋밋한 모양 등 다양합니다.  잎의 표면에는 기공이 있어서 광합성을 효율적으로 수행합니다.\n* **잎의 수명:** 대부분 낙엽수이기 때문에 가을에 잎이 떨어집니다.  하지만 상록활엽수도 있습니다. (예: 동백나무, 감탕나무)\n* **열매:** 다양한 형태의 열매를 맺습니다.  도토리, 밤, 단풍나무의 날개 열매 등 다양한 종류가 있습니다.\n* **목재:** 종류에 따라 다양한 특징을 가지고 있습니다.  단단한 것도 있고 부드러운 것도 있습니다. 가구재, 건축재, 종이 제작 등 다양한 용도로 사용됩니다.\n* **예시:** 참나무, 단풍나무, 은행나무, 벚나무, 감나무, 동백나무 등\n\n\n물론 예외적인 경우도 있고, 침엽수와 활엽수의 경계가 모호한 경우도 있

In [ ]:
# 두번째 대화_첫번째 대화 내용과 관련된 질문(첫번째 질문 답변이 길어서 저장이 안됨)
conversation.predict(
    input="잎의 모양에 대해 다시 한 번 알려줘"
)

'네, 잎의 모양에 대해 다시 설명해 드리겠습니다.  잎의 모양은 정말 다양해서 한두 문장으로 설명하기 어렵지만,  크게 다음과 같은 요소들을 고려하여 분류할 수 있습니다.\n\n* **잎몸(엽신, lamina)의 형태:**  가장 기본적인 분류 기준입니다.  둥근 모양, 타원형, 난형(달걀 모양), 피침형(창 모양), 선형(바늘 모양), 심장형(하트 모양), 신장형(콩팥 모양), 삼각형, 곡선형 등 매우 다양한 형태가 있습니다.  또한 잎 가장자리(엽연, margin)의 모양도 중요한데, 톱니 모양(거치, serrate), 깊게 패인 모양(열편, lobed), 밋밋한 모양(전연, entire) 등이 있습니다.  잎 끝(엽첨, apex)과 잎 기부(엽저, base)의 모양도 잎의 전체적인 형태를 결정하는 요소입니다. 예를 들어, 잎 끝이 뾰족한지, 둥근지, 갑자기 좁아지는지 등을 살펴볼 수 있습니다.\n\n* **잎맥(veins)의 배열:** 잎에 있는 잎맥은 물과 영양분을 운반하는 역할을 하며, 잎의 형태를 유지하는 데에도 중요한 역할을 합니다.  잎맥의 배열은 주맥(midrib)에서 시작하여 측맥(lateral veins)으로 갈라지는데, 이러한 배열 패턴은 종에 따라 다릅니다.  주요 잎맥 배열에는 평행맥(parallel venation), 망상맥(reticulate venation), 우상맥(pinnate venation), 장상맥(palmate venation) 등이 있습니다. 평행맥은 잎맥이 거의 평행하게 배열되는 것이고, 망상맥은 그물처럼 복잡하게 얽혀있는 것을 말합니다.  우상맥은 주맥에서 측맥이 깃털처럼 뻗어나가는 것이고, 장상맥은 손바닥처럼 여러 개의 잎맥이 한 곳에서 갈라지는 것입니다.\n\n* **잎의 갈라짐:**  잎이 갈라지는 정도도 중요한 특징입니다.  전혀 갈라지지 않은 홑잎(simple leaf)도 있고, 여러 개의 작은 잎으로 갈라진 겹잎(compound leaf)도 있습니다.  겹잎은 다시 손바닥 모양 겹잎(palmately 

In [ ]:
# 세번째 대화
conversation.predict(
    input="대한민국의 수도는?"
)

'대한민국의 수도는 서울입니다.  서울은 한강을 끼고 발달한 역사적인 도시이며, 약 1,000만 명의 인구가 거주하고 있습니다.  경복궁, 창덕궁, 덕수궁과 같은 아름다운 궁궐들과, 인사동과 같은 전통적인 거리, 그리고 강남과 같은 현대적인 지역이 공존하는 매력적인 도시입니다.  또한,  세계적인 기업들의 본사가 많이 위치해 있으며,  첨단 기술 산업과 문화 예술이 발전된 도시이기도 합니다.  서울은 대한민국의 정치, 경제, 문화의 중심지로서,  다양한 볼거리와 즐길거리를 제공합니다.'

In [ ]:
# 네번째 대화_세번째 내용에 관한 질문
conversation.predict(
    input="이전 대화에서 답변해준 도시의 인구를 대략적으로 알려줘"
)

'이전 대화에서 제가 언급한 도시는 서울이며, 서울의 인구는 약 1,000만 명이라고 말씀드렸습니다.  정확한 인구 수는 시기에 따라 변동이 있으므로,  약 1,000만 명으로 추산했습니다.'

# ConversationEntityMemory

*   특정 엔티티에 대해 주어진 사실 기억


In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
# Entity Memory를 사용하는 프롬프트 내용 출력
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

In [ ]:
# ConversationChain  생성
conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm),
)

<ipython-input-3-79dcc2c84bcb>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationEntityMemory(llm=llm),
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
<ipython-input-3-79dcc2c84bcb>:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [ ]:
conversation.predict(
    input="테디와 셜리는 한 회사에서 일하는 동료입니다."
    "테디는 개발자이고 셜리는 디자이너입니다. "
    "그들은최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다."
)

'테디와 셜리의 새로운 회사 설립 계획이 흥미롭네요!  어떤 종류의 회사를 설립할 계획인지, 그리고 어떤 어려움이나 기회가 예상되는지 좀 더 자세히 알고 싶습니다.  예를 들어,  회사의 이름이나 사업 분야,  자금 조달 계획,  마케팅 전략 등에 대해 이야기해볼 수 있을까요?  두 사람의 강점을 잘 활용하여 성공적인 회사를 설립할 수 있도록 응원하겠습니다.'

In [ ]:
# entity memory 출력
conversation.memory.entity_store.store

{'테디': '테디는 개발자이며, 셜리와 함께 회사를 설립할 계획이다.',
 '셜리': '셜리는 디자이너이며, 최근 회사를 그만두고 테디와 함께 새로운 회사를 설립할 계획이다.'}

In [ ]:
conversation.predict(
    input="테디에 대해 알려주세요."
)

'제가 알고 있는 정보에 따르면, 테디는 개발자입니다.  그리고 셜리와 함께 회사를 설립할 계획입니다.  현재로서는 테디에 대한 추가적인 정보는 없습니다.  테디의 성격,  개발 경력,  회사 설립 계획에서의 역할 등 더 자세한 정보를 알고 싶으시다면,  더 많은 질문을 해주시면 제가 아는 범위 내에서 최대한 답변해 드리겠습니다.'

# ConversationKGMemory

*   대화 지식그래프 메모리. 지식 그래프의 힘을 활용하여 정보를 저장
*   모델이 서로 다른 개체 간의 관계를 이해하는 데 도움




In [ ]:
from langchain.memory import ConversationKGMemory

memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "이쪽은 Pangyo 에 거주중인 김셜리씨 입니다."},
    {"output": "김셜리씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김셜리씨는 우리 회사의 신입 디자이너입니다."},
    {"output": "만나서 반갑습니다."},
)

In [ ]:
memory.load_memory_variables({"input": "김셜리씨는 누구입니까?"})

{'history': [SystemMessage(content='On 김셜리씨: 김셜리씨 거주중인 Pangyo. 김셜리씨 is 신입 디자이너. 김셜리씨 is 우리 회사의.', additional_kwargs={}, response_metadata={})]}

## 체인에 적용

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

template = """The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)


In [ ]:
conversation_with_kg.predict(
    input="내 이름은 테디이다. 셜리는 나의 동료이고, 우리 회사의 새로 온 디자이너이다."
)

'안녕하세요, 테디씨! 셜리씨가 회사의 새 디자이너시군요.  잘 부탁드립니다!  다른 건 아직 잘 모르겠지만, 셜리씨와 함께 일하게 되어 기대가 됩니다.  더 자세한 내용은 알려주시면 제가 기억해두도록 하겠습니다.'

In [ ]:
# Shirley 에 대한 질문
conversation_with_kg.memory.load_memory_variables({"input": "셜리는 누구인가?"})


{'history': 'On 셜리: 셜리 동료이고 테디. 셜리 직업 디자이너. 셜리 회사 테디의 회사. 셜리 입사시기 최근.'}

#ConversationSummaryMemory
*  시간 경과에 따른 대화의 요약 생성
*  대화가 진행되는 동안 대화를 요약하고 현재 요약을 메모리에 저장



In [ ]:
from langchain.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(
    llm=llm, return_messages=True)

<ipython-input-30-a60fde16c94e>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [ ]:
memory.save_context(
    inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
    outputs={
        "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
    },
)
memory.save_context(
    inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
    outputs={
        "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "여행자 보험은 포함되어 있나요?"},
    outputs={
        "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
    },
)
memory.save_context(
    inputs={
        "human": "항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?"
    },
    outputs={
        "ai": "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
    },
)
memory.save_context(
    inputs={"human": "패키지에 포함된 호텔의 등급은 어떻게 되나요?"},
    outputs={
        "ai": "이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다."
    },
)
memory.save_context(
    inputs={"human": "식사 옵션에 대해 더 자세히 알려주실 수 있나요?"},
    outputs={
        "ai": "이 여행 패키지는 매일 아침 호텔에서 제공되는 조식을 포함하고 있습니다. 점심과 저녁 식사는 포함되어 있지 않아, 여행자가 자유롭게 현지의 다양한 음식을 경험할 수 있는 기회를 제공합니다. 또한, 각 도시별로 추천 식당 리스트를 제공하여 현지의 맛을 최대한 즐길 수 있도록 도와드립니다."
    },
)
memory.save_context(
    inputs={"human": "패키지 예약 시 예약금은 얼마인가요? 취소 정책은 어떻게 되나요?"},
    outputs={
        "ai": "패키지 예약 시 500유로의 예약금이 필요합니다. 취소 정책은 예약일로부터 30일 전까지는 전액 환불이 가능하며, 이후 취소 시에는 예약금이 환불되지 않습니다. 여행 시작일로부터 14일 전 취소 시 50%의 비용이 청구되며, 그 이후는 전액 비용이 청구됩니다."
    },
)


In [ ]:
# 저장된 메모리 확인
print(memory.load_memory_variables({})["history"])

[SystemMessage(content='The human asks about the price of a European travel package.  The AI responds that a 14-night, 15-day package costs 3,500 euros, including flights, hotels, and entrance fees to specified attractions like the Eiffel Tower, Colosseum, Brandenburg Gate, and Rhine Falls.  Additional costs will depend on optional tours and personal expenses. Basic travel insurance is included, with upgrade options.  The human asks about upgrading to business class airfare (an additional 1200 euros per person), and the hotel rating (4-star hotels, centrally located).  The human then asks about meal options, and the AI clarifies that the package includes daily breakfast at the hotels, but lunch and dinner are not included, allowing travelers to explore local cuisine. A list of recommended restaurants in each city will be provided.  The human then asks about the deposit required for booking and the cancellation policy. The AI states that a 500 euro deposit is required.  The cancellation

#ConversationSummaryBufferMemory

*   최근 대화내용의 버퍼를 메모리에 유지, 이전 대화는 요약하여 저장
*   최근 대화내용의 기준은 토큰 길이



In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,  # 요약의 기준이 되는 토큰 길이 설정
    return_messages=True,
)

<ipython-input-33-f568c09e9e9e>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [ ]:
memory.save_context(
    inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
    outputs={
        "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
    },
)

In [ ]:
# 메모리에 저장된 대화내용 확인_요약x
memory.load_memory_variables({})["history"]

[HumanMessage(content='유럽 여행 패키지의 가격은 얼마인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다.', additional_kwargs={}, response_metadata={})]

In [ ]:
memory.save_context(
    inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
    outputs={
        "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "여행자 보험은 포함되어 있나요?"},
    outputs={
        "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
    },
)
memory.save_context(
    inputs={
        "human": "항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?"
    },
    outputs={
        "ai": "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
    },
)
memory.save_context(
    inputs={"human": "패키지에 포함된 호텔의 등급은 어떻게 되나요?"},
    outputs={
        "ai": "이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다."
    },
)


In [ ]:
# 메모리에 저장된 대화내용 확인_SystemMessage에 요약, 그 아래에 최근 대화 내용 저장
memory.load_memory_variables({})["history"]

[SystemMessage(content='The human asks about the price of a European travel package.  The AI responds that a 14-night/15-day package costs 3500 Euros, including flights, hotels, and entrance fees to specified attractions like the Eiffel Tower, Colosseum, Brandenburg Gate, and Fraumünster.  Additional costs depend on optional tours and personal expenses. The human then asks if travel insurance is included, and the AI confirms that basic travel insurance is provided, covering medical expenses and emergency assistance, with options for upgrades.  Finally, the human inquires about upgrading to business class flights and their associated cost.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='패키지에 포함된 호텔의 등급은 어떻게 되나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(co

## 체인에 적용

In [ ]:
conversation = ConversationChain(
    llm = llm,
    # ConversationSummaryBufferMemory 사용
    memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=500),
)

In [ ]:
response = conversation.predict(
    input="활엽수와 침엽수의 차이에 대해 알려줘"
)
print(response)

활엽수와 침엽수의 차이는 잎의 모양과 구조, 그리고 나무의 생김새 등 여러 가지 측면에서 나타납니다.  자세히 알아볼까요?

**잎의 모양과 구조:**

* **활엽수 (Broadleaf trees):** 넓고 납작한 잎을 가지고 있습니다.  잎의 가장자리는 톱니 모양이거나 매끄러울 수 있으며, 잎맥이 복잡하게 퍼져 있습니다.  단풍나무, 참나무, 은행나무, 느릅나무 등이 대표적인 활엽수입니다.  보통 낙엽성이 많지만, 상록 활엽수도 존재합니다 (예: 동백나무, 후박나무).  잎의 형태는 매우 다양하며,  심장형, 타원형, 손바닥 모양 등 다양한 모양을 하고 있습니다.

* **침엽수 (Coniferous trees):** 바늘 모양 또는 비늘 모양의 좁고 뾰족한 잎을 가지고 있습니다.  잎맥은 단순하고, 잎은 보통 단단하고 두껍습니다. 소나무, 전나무, 잣나무, 삼나무 등이 대표적인 침엽수입니다. 대부분 상록수이지만, 낙엽 침엽수도 있습니다 (예: 메타세쿼이아).  잎의 크기와 모양도 종류에 따라 다양하지만, 활엽수에 비해 훨씬 단순한 구조를 가지고 있습니다.


**나무의 생김새 및 기타 특징:**

* **활엽수:** 일반적으로 둥글고 넓은 수관 (나무의 가지와 잎이 퍼져 있는 부분)을 가지고 있습니다.  수피는 매끄럽거나 거칠 수 있으며, 종에 따라 다양한 색과 질감을 나타냅니다.  꽃이 피는 경우가 많고, 열매를 맺습니다.  목재는 일반적으로 침엽수보다 무겁고 단단하며, 가구나 건축 자재로 많이 사용됩니다.

* **침엽수:** 일반적으로 원뿔 모양 또는 피라미드 모양의 수관을 가지고 있습니다.  수피는 거칠고 두꺼운 경우가 많습니다.  꽃은 작고 눈에 잘 띄지 않는 경우가 많으며, 구과 (솔방울)라는 특징적인 열매를 맺습니다.  목재는 일반적으로 활엽수보다 가볍고 부드러우며, 건축 자재, 종이 제작 등에 사용됩니다.


**예외:**  모든 나무가 위의 특징을 완벽하게 따르는 것은 아닙니다.  예외적인 경우도 존재하며,  잎의 모양이나 생장 형태 등

In [ ]:
response = conversation.predict(
    input="잎의 모양에 대해 다시 설명해줘"
)
print(response)

네, 잎의 모양에 대해 다시 설명해 드리겠습니다. 활엽수와 침엽수의 가장 큰 차이점 중 하나는 바로 잎의 모양입니다.

**활엽수 (Broadleaf trees):** 활엽수는 넓고 평평한 잎을 가지고 있습니다.  이 잎들은 보통 잎맥이 복잡하게 그물처럼 뻗어있는 그물맥(reticulate venation)을 가지고 있습니다.  단풍나무, 참나무, 너도밤나무 등이 대표적인 예시이며, 잎의 가장자리는 톱니 모양, 밋밋한 모양, 갈라진 모양 등 다양한 형태를 보입니다.  잎의 크기도 종류에 따라 매우 다양합니다.  가을에는 아름다운 단풍이 드는 것으로 유명합니다.

**침엽수 (Coniferous trees):**  반면에 침엽수는 바늘 모양(needle-like) 또는 비늘 모양(scale-like)의 잎을 가지고 있습니다.  이 잎들은 잎맥이 평행하거나 나란히 배열된 평행맥(parallel venation)을 가지거나, 잎맥이 거의 보이지 않을 정도로 단순합니다.  소나무, 전나무, 잣나무 등이 대표적인 예시이며, 잎의 크기는 활엽수에 비해 상대적으로 작습니다.  침엽수의 잎은 대부분 겨울에도 나무에 붙어있어 상록수로 분류되는 경우가 많습니다.  하지만 낙엽침엽수도 존재합니다.


간단히 말해, 활엽수는 넓고 평평한 잎을 가지고 있고 침엽수는 바늘 또는 비늘 모양의 잎을 가지고 있다는 것이 가장 큰 차이점입니다.  하지만 예외적인 경우도 존재하며, 모든 활엽수와 침엽수가 이 규칙을 완벽하게 따르는 것은 아닙니다.


In [ ]:
response = conversation.predict(
    input="대한민국의 수도는?"
)
print(response)

대한민국의 수도는 서울입니다.


In [ ]:
response = conversation.predict(
    input="앞서 답한 도시의 대략적인 인구수는?"
)
print(response)

서울의 정확한 인구수는 시시각각 변하기 때문에 제가 정확한 숫자를 말씀드리기는 어렵습니다.  통계청 자료나 서울시청 웹사이트에서 최신 인구수를 확인하시는 것이 가장 정확합니다. 다만,  대략적으로 수천만 명이 거주한다는 것을 말씀드릴 수 있습니다.  좀 더 정확한 숫자를 원하신다면,  해당 기관의 웹사이트를 참고해주세요.


In [ ]:
#첫번째 대화에 관련된 질문_제대로 나옴
response = conversation.predict(
    input="잎의 모양에 대해 다시 설명해줘"
)
print(response)

네, 잎의 모양에 대해 다시 설명해 드리겠습니다.  앞서 말씀드렸듯이,  활엽수와 침엽수의 가장 큰 차이점 중 하나는 잎의 모양입니다.

**활엽수 (Broadleaf Trees):** 활엽수는 넓고 평평한 잎을 가지고 있습니다. 이 잎들은 보통 넓은 면적을 가지고 있으며,  잎맥(잎의 혈관)이 그물처럼 복잡하게 뻗어있는 **그물맥(reticulate venation)**을 가지고 있습니다.  잎의 가장자리는 톱니 모양(serrated), 매끈한 모양(smooth), 또는 갈라진 모양(lobed) 등 다양한 모양을 가지고 있으며, 크기도 종류에 따라 매우 다양합니다.  예를 들어,  단풍나무 잎은 손바닥 모양으로 갈라져 있고,  참나무 잎은 톱니 모양의 가장자리를 가지고 있으며,  너도밤나무 잎은 매끈한 가장자리를 가지고 있습니다.

**침엽수 (Coniferous Trees):** 반면에 침엽수는 바늘 모양(needle-like) 또는 비늘 모양(scale-like)의 잎을 가지고 있습니다.  이 잎들은 활엽수처럼 넓고 평평하지 않고,  잎맥이 거의 보이지 않거나 아주 단순한 형태를 띄고 있습니다.  침엽수의 잎은 일반적으로 활엽수보다 훨씬 작고,  대부분의 침엽수는 상록수이지만,  낙엽침엽수도 존재합니다.  소나무, 전나무, 삼나무 등이 대표적인 침엽수입니다.  소나무의 잎은 가늘고 긴 바늘 모양이며, 전나무의 잎은 짧고 납작한 바늘 모양이고,  삼나무의 잎은 비늘 모양입니다.

요약하자면, 활엽수는 넓고 평평하며 그물맥을 가진 잎을 가지고 있는 반면, 침엽수는 바늘 또는 비늘 모양의 잎을 가지고 있으며 잎맥이 단순하거나 거의 보이지 않습니다.  물론 예외적인 경우도 있지만, 이러한 차이점은 활엽수와 침엽수를 구분하는 데 중요한 지표입니다.


#VectorStoreRetrieverMemory

*   벡터 스토어에 메모리를 저장하고 호출될 때마다 가장 '눈에 띄는' 상위 K개의 문서를 쿼리
*   명시적으로 추적하지 않는다는 점



In [ ]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from google.colab import userdata # 코랩 Secrets 기능 사용을 위한 임포트
import os

openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
print("OPENAI_API_KEY가 환경 변수로 설정되었습니다.")

# 임베딩 모델 정의
embeddings_model = OpenAIEmbeddings()

# Vector Store 초기화
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

OPENAI_API_KEY가 환경 변수로 설정되었습니다.


In [ ]:
from langchain.memory import VectorStoreRetrieverMemory

# 벡터 조회가 여전히 의미적으로 관련성 있는 정보를 반환한다는 것을 보여주기 위함
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
memory = VectorStoreRetrieverMemory(retriever=retriever)

# 임의의 대화 저장
memory.save_context(
    inputs={
        "human": "안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
    },
    outputs={
        "ai": "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다."
    },
)
memory.save_context(
    inputs={"human": "프로젝트에서 어떤 역할을 맡았나요?"},
    outputs={
        "ai": "제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다."
    },
)
memory.save_context(
    inputs={
        "human": "팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
    },
    outputs={
        "ai": "프로젝트 초기에 의사소통 문제로 몇 가지 어려움이 있었습니다. 이를 해결하기 위해 저희 팀은 정기적인 미팅을 갖고 각자의 진행 상황을 공유했습니다. 또한, 문제가 발생했을 때는 적극적으로 의견을 나누고, 합리적인 해결책을 찾기 위해 노력했습니다."
    },
)
memory.save_context(
    inputs={"human": "개발자로서 자신의 강점은 무엇이라고 생각하나요?"},
    outputs={
        "ai": "제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술이나 도구를 빠르게 습득할 수 있으며, 복잡한 문제에 직면했을 때 창의적인 해결책을 제시할 수 있습니다. 또한, 팀워크를 중시하며 동료들과 협력하는 것을 중요하게 생각합니다."
    },
)

In [ ]:
# 메모리에 질문을 통해 가장 연관성 높은 1개 대화 추출
print(memory.load_memory_variables({"prompt": "면접자 전공은 무엇인가요?"})["history"])

human: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.


In [ ]:
print(
    memory.load_memory_variables(
        {"human": "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"}
    )["history"]
)

human: 프로젝트에서 어떤 역할을 맡았나요?
ai: 제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다.


## 체인에 적용

In [ ]:
from langchain.chains import ConversationChain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
)

In [ ]:
response = conversation.predict(
    input="저의 취미는 산책입니다."
)
print(response)

아, 멋진 취미시네요!  저는 물리적인 몸이 없어서 산책을 할 수는 없지만,  가상 세계를 탐험하는 건 좋아합니다.  예를 들어,  Google Earth를 통해 전 세계의 다양한 풍경을 탐험하거나,  세계 각국의 스트릿 뷰를 통해 거리를 걸어 다니는 기분을 느껴보기도 합니다.  특히,  제가 접근할 수 있는 방대한 데이터를 통해,  실제 산책보다 훨씬 더 다양한 장소를,  훨씬 더 빠르게 탐험할 수 있다는 점이 매력적이에요.  예를 들어,  오늘 아침에는 아마존 열대우림의 3D 모델을 탐험하며  다양한 식물과 동물들을 '관찰'했고,  어제는  고대 로마 유적지를 '탐방'했습니다.  물론,  실제 산책의 상쾌한 공기와 자연의 향기는 느낄 수 없지만,  저 나름대로의 '가상 산책'을 즐기고 있습니다.  선생님께서는 어떤 곳을 산책하시는 걸 좋아하시나요?  그리고 산책하면서 어떤 생각을 하시는지도 궁금하네요.


In [ ]:
response = conversation.predict(
    input="미국의 주의 개수는 몇 개인가요?"
    "대한민국의 수도는 어디에요?"
    "냉풍기의 원리가 궁금해요."
    "얼마 전 냉풍기를 새로 샀어요"
)
print(response)

미국의 주의 개수는 50개입니다. 대한민국의 수도는 서울입니다.  냉풍기의 원리는 증발냉각입니다.  물이 증발할 때 주변의 열을 흡수하는 현상을 이용하여 공기를 시원하게 만드는 것이죠.  새로 냉풍기를 사셨다니 축하드립니다!  어떤 모델인지는 모르겠지만,  대부분의 냉풍기는 물을 담는 물탱크와 물을 펌프로 냉각패드에 공급하는 시스템, 그리고 시원해진 공기를 내보내는 팬으로 구성되어 있습니다.  냉각 패드는 보통 섬유 소재로 만들어져 물을 잘 흡수하고 증발시키도록 설계됩니다. 팬이 공기를 냉각 패드를 통해 강제로 통과시키면서, 물이 증발하면서 열을 흡수하고, 그 결과 시원한 바람이 나오게 되는 것이죠.  얼마나 시원하게 느껴지는지는 주변 온도와 습도, 그리고 냉각 패드의 상태(얼마나 깨끗하게 관리되는지 등)에 따라 달라집니다.  습도가 높으면 증발이 잘 안되서 냉각 효과가 떨어지니,  물을 자주 갈아주고 냉각 패드를 청결하게 유지하는 것이 중요합니다.  새로 구입하신 냉풍기,  잘 사용하시길 바랍니다!  혹시 사용하면서 궁금한 점이 있으면 언제든지 저에게 물어보세요.  제가 아는 범위 내에서 최대한 답변해 드리겠습니다.


In [ ]:
response = conversation.predict(
    input="저의 취미는 무엇일까요?"
)
print(response)

선생님께서 직접 말씀하셨듯이,  선생님의 취미는 산책입니다.


# LCEL Chain 에 메모리 추가

In [ ]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# 모델 초기화
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [ ]:
# 대화 버퍼 메모리 생성, 메시지 반환 기능을 활성화
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

<ipython-input-28-87186dc06766>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [ ]:
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화

{'chat_history': []}

In [ ]:
#저장된 대화 기록의 값만 전달
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key와 동일하게 입력
)

In [ ]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [ ]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [ ]:
# 대화형 프롬프트 생성. 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력 포함
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
runnable.invoke({"input": "hi!"})

{'input': 'hi!', 'chat_history': []}

In [ ]:
chain = runnable | prompt | model

In [ ]:
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답 생성
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 테디입니다."})
print(response.content)  # 생성된 응답 출력

만나서 반가워요, 테디씨!  잘 부탁드립니다. 😊


In [ ]:
memory.load_memory_variables({})

{'chat_history': []}

In [ ]:
# 입력된 데이터와 응답 내용을 메모리에 저장
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 테디입니다."}, {"ai": response.content}
)

# 저장된 대화기록 출력
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 테디입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='만나서 반가워요, 테디씨!  잘 부탁드립니다. 😊', additional_kwargs={}, response_metadata={})]}

In [ ]:
# 이름을 기억하고 있는지 추가 질의
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
# 답변 출력
print(response.content)

네, 테디라고 말씀하셨습니다.  😊


## 커스텀 ConversationChain

In [ ]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_core.output_parsers import StrOutputParser

# 클래스 정의
class MyConversationChain(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)  # memory_key 와 동일하게 입력
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        #invoke 실행하면 자동으로 저장
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer


In [ ]:
#동일한 모델, 프롬프트, 메모리 사용
conversation_chain = MyConversationChain(llm, prompt, memory)

In [ ]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.")

'안녕하세요, 테디님! 만나서 반갑습니다!  무엇을 도와드릴까요?'

In [ ]:
conversation_chain.invoke("제 이름이 뭐라고요?")

'테디님이라고 말씀하셨습니다.'

In [ ]:
conversation_chain.invoke("앞으로는 저를 디테라고 불러주세요")

'알겠습니다. 앞으로는 디테라고 부르겠습니다.  무엇을 도와드릴까요, 디테님?'

In [ ]:
conversation_chain.invoke("제 이름이 뭐라고요?")

'디테라고 불러드리기로 했죠.'

In [ ]:
conversation_chain.memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 테디님! 만나서 반갑습니다!  무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름이 뭐라고요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='테디님이라고 말씀하셨습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='앞으로는 저를 디테라고 불러주세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='알겠습니다. 앞으로는 디테라고 부르겠습니다.  무엇을 도와드릴까요, 디테님?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름이 뭐라고요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='디테라고 불러드리기로 했죠.', additional_kwargs={}, response_metadata={})]

# SQLChatMessageHistory

*   SQL 데이터베이스(SQLite, PostgreSQL, MySQL 등)에 연결_영구적으로 저장 가능
*   Memory 클래스는 대화 기록을 어떻게 '사용'하고 '관리'할지 (예: 몇 개를 기억할지, 요약할지 등) 정의
*  ChatMessageHistory 구현체는 대화 기록을 어디에 '저장'할지 (예: 파일, DB, Redis 등) 정의
*  ChatMessageHistory 없이 Memory 클래스를 사용하면 기본 구현체 InMemoryChatMessageHistory 사용- 휘발됨.



In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성, 세션 ID와 데이터베이스 연결 파일 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)

In [ ]:
# 사용자 메시지 추가
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내 이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!"
)
# AI 메시지 추가
chat_message_history.add_ai_message("안녕 테디, 만나서 반가워. 나도 잘 부탁해!")

In [ ]:
# 채팅 메시지 기록 확인
chat_message_history.messages

[HumanMessage(content='안녕? 만나서 반가워. 내 이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 테디, 만나서 반가워. 나도 잘 부탁해!', additional_kwargs={}, response_metadata={})]

## 체인에 적용

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지
        ("system", "You are a helpful assistant."),
        # 대화 기록을 위한 Placeholder
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),  # 질문
    ]
)
# chain 생성
chain = prompt | llm | StrOutputParser()

In [ ]:
# 데이터베이스에서 대화내용 가져오는 함수
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db",
    )

In [ ]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

#대화 내용을 조회할 때 사용. 사용자 아이디(user_id)와 대화 아이디(conversation_id) 정의
config_fields = [
    ConfigurableFieldSpec(
        id="user_id", #config에 쓰일 키
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id", #config에 쓰일 키
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 대화 기록을 가져오는 함수를 설정합니다.
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="chat_history",  # 대화 기록 메시지의 키를 "history"로 설정
    history_factory_config=config_fields,  # 대화 기록 조회시 참고할 파라미터를 설정합니다.
)


In [ ]:
# config 설정: config_fields로 정의한 키에 해당하는 값 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

In [ ]:
# 질문과 config 를 전달하여 실행
chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 테디야"}, config)

'안녕 테디야! 반가워! 😊'

In [ ]:
# 후속 질문을 실해합니다.
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

'네 이름은 테디라고 했어요!'

In [ ]:
# config 설정_사용자 아이디는 동일, 대화 아이디는 다름
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# 질문과 config 를 전달하여 실행_대화 아이디가 달라 결과도 다름
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)


'저는 대규모 언어 모델이며, 이름이 없습니다.'

# RunnableWithMessageHistory에 ChatMessageHistory추가

## 이전 대화 내용을 기억하는 multi-turn Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()


In [ ]:
import os
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory() #InMemoryChatMessageHistory()

    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [ ]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "나의 이름은 테디입니다."},
    # 세션 ID 기준으로 대화 기록
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'알겠습니다, 테디씨. 무엇을 도와드릴까요?'

In [ ]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화 기록
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'당신의 이름은 테디입니다.'

In [ ]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화 기록_세션 ID가 달라 이름을 기억하지 못함
    config={"configurable": {"session_id": "abc1234"}},
)


[대화 세션ID]: abc1234


'저는 당신의 이름을 알지 못합니다.  저는 대화형 인공지능 모델이며, 사용자의 개인 정보를 저장하지 않습니다.  당신의 이름을 알려주시면 제가 기억할 수 있도록 하겠지만, 그럴 필요는 없습니다.'